In [1]:
#!conda activate tensorflow
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("Reportes/Nuevos/datos_abiertos_covid19/COVID19_Mexico_13.04.2020_cod.csv", sep=',', index_col=0)
data.head(7)

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
FECHA_ACTUALIZACION,,,,,,,,,,,,,,,,,,,,,
2020-04-13,2,9,15,1,15,15,37,2,2020-04-09,2020-03-28,...,2,2,2,2,2,1,99,99,97,1
2020-04-13,2,9,28,2,16,28,32,1,2020-04-06,2020-04-04,...,2,2,2,2,1,1,99,99,97,97
2020-04-13,1,12,15,2,15,15,31,2,2020-04-06,2020-04-04,...,2,2,2,2,2,2,99,99,97,2
2020-04-13,1,4,2,2,21,2,4,2,2020-04-02,2020-03-26,...,2,1,2,2,99,2,99,México,97,2
2020-04-13,1,12,9,2,99,9,12,2,2020-03-28,2020-03-17,...,2,2,2,2,2,1,99,México,97,2
2020-04-13,1,12,9,1,99,9,14,1,2020-04-01,2020-03-26,...,2,2,2,2,1,1,99,México,97,97
2020-04-13,1,12,12,1,12,12,56,2,2020-04-12,2020-04-05,...,2,2,2,2,2,3,99,99,97,1


In [3]:
print(data.columns)
print(data.shape)
registers = data.shape[0]
print(registers)

Index(['ORIGEN', 'SECTOR', 'ENTIDAD_UM', 'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES',
       'MUNICIPIO_RES', 'TIPO_PACIENTE', 'FECHA_INGRESO', 'FECHA_SINTOMAS',
       'FECHA_DEF', 'INTUBADO', 'NEUMONIA', 'EDAD', 'NACIONALIDAD', 'EMBARAZO',
       'HABLA_LENGUA_INDI', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_CON', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'RESULTADO', 'MIGRANTE',
       'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')
(38670, 33)
38670


In [4]:
data['FECHA_INGRESO'] = pd.to_datetime(data['FECHA_INGRESO'], format='%Y/%m/%d')
data['FECHA_SINTOMAS'] = pd.to_datetime(data['FECHA_SINTOMAS'], format='%Y/%m/%d')
data['FECHA_DEF'] = data['FECHA_DEF'].replace('9999-99-99',np.NaN)
data['FECHA_DEF'] = pd.to_datetime(data['FECHA_DEF'], format='%Y/%m/%d')

#data['FECHA_DEF'].replace('9999-99-99',np.NaN)
#pd.to_datetime(data["FECHA_DEF"], format='%d/%m/%Y')
#data.isnull().any()
#data['FECHA_DEF'] = data['FECHA_DEF'] == '9999-99-99' ? '' : data['FECHA_DEF']

In [5]:
xl_file = pd.ExcelFile("Reportes/Nuevos/diccionario_datos_covid19/Catalogos_0412.xlsx")
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [6]:
origen = dfs['Catálogo ORIGEN'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
sector = dfs['Catálogo SECTOR'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
sexo = dfs['Catálogo SEXO'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
tipo_paciente = dfs['Catálogo TIPO_PACIENTE'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
si_no = dfs['Catálogo SI_NO'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
nacionalidad = dfs['Catálogo NACIONALIDAD'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
resultado = dfs['Catálogo RESULTADO'][1:].set_index('Unnamed: 0').to_dict()['Unnamed: 1']
entidades = dfs['Catálogo de ENTIDADES'].set_index('CLAVE_ENTIDAD')['ABREVIATURA']
municipios = dfs['Catálogo MUNICIPIOS'].set_index(['CLAVE_ENTIDAD','CLAVE_MUNICIPIO']).to_dict()['MUNICIPIO']

In [7]:
#result = list(data[['ENTIDAD_RES','MUNICIPIO_RES']].to_records(index=False))

def replace_touple(df, col, dicc, colOut):
    salida = []
    salida = list()
    for key1, key2 in df[col]:
        if (key1,key2) in dicc:
            salida.append(dicc[(key1,key2)])
        else:
            salida.append(np.NaN)
       
    df[colOut] = salida

#tmp = [v in municipios for v in dataf['ENTIDAD_MUNICIPIO_RES']] 
#dataf.loc[tmp]

#dataf['ENTIDAD_MUNICIPIO_RES'].mask(dataf['ENTIDAD_MUNICIPIO_RES'] == (15,37), (1,1)), inplace=True)

In [8]:
dataf = data.copy()
dataf['ENTIDAD_MUNICIPIO_RES'] = list(zip(dataf['ENTIDAD_RES'], dataf['MUNICIPIO_RES']))
replace_touple(dataf, 'ENTIDAD_MUNICIPIO_RES', municipios, 'MUNICIPIO_RES')

dataf = dataf.replace({'ORIGEN': origen})
dataf = dataf.replace({'SECTOR': sector})
dataf = dataf.replace({'ENTIDAD_UM': entidades})
dataf = dataf.replace({'SEXO': sexo})
dataf = dataf.replace({'ENTIDAD_NAC': entidades})
dataf = dataf.replace({'ENTIDAD_RES': entidades})
dataf = dataf.replace({'TIPO_PACIENTE': tipo_paciente})
dataf = dataf.replace({'INTUBADO': si_no})
dataf = dataf.replace({'NEUMONIA': si_no})
dataf = dataf.replace({'NACIONALIDAD': nacionalidad})
dataf = dataf.replace({'EMBARAZO': si_no})
dataf = dataf.replace({'HABLA_LENGUA_INDI': si_no})
dataf = dataf.replace({'DIABETES': si_no})
dataf = dataf.replace({'EPOC': si_no})
dataf = dataf.replace({'ASMA': si_no})
dataf = dataf.replace({'INMUSUPR': si_no})
dataf = dataf.replace({'HIPERTENSION': si_no})
dataf = dataf.replace({'OTRA_CON': si_no})
dataf = dataf.replace({'CARDIOVASCULAR': si_no})
dataf = dataf.replace({'OBESIDAD': si_no})
dataf = dataf.replace({'RENAL_CRONICA': si_no})
dataf = dataf.replace({'TABAQUISMO': si_no})
dataf = dataf.replace({'OTRO_CASO': si_no})
dataf = dataf.replace({'RESULTADO': resultado})
dataf = dataf.replace({'MIGRANTE': si_no})
#dataf = dataf.replace({'PAIS_NACIONALIDAD': origen})
#dataf = dataf.replace({'PAIS_ORIGEN': origen})
dataf = dataf.replace({'UCI': si_no})

dataf.head()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,ENTIDAD_MUNICIPIO_RES
FECHA_ACTUALIZACION,,,,,,,,,,,,,,,,,,,,,
2020-04-13,FUERA DE USMER,PRIVADA,MC,MUJER,MC,MC,HUIXQUILUCAN,HOSPITALIZADO,2020-04-09,2020-03-28,...,NO,NO,NO,NO,Positivo SARS-CoV-2,NO ESPECIFICADO,99,97,SI,"(15, 37)"
2020-04-13,FUERA DE USMER,PRIVADA,TS,HOMBRE,MN,TS,REYNOSA,AMBULATORIO,2020-04-06,2020-04-04,...,NO,NO,NO,SI,Positivo SARS-CoV-2,NO ESPECIFICADO,99,97,NO APLICA,"(28, 32)"
2020-04-13,USMER,SSA,MC,HOMBRE,MC,MC,CHIMALHUACÁN,HOSPITALIZADO,2020-04-06,2020-04-04,...,NO,NO,NO,NO,No positivo SARS-CoV-2,NO ESPECIFICADO,99,97,NO,"(15, 31)"
2020-04-13,USMER,IMSS,BC,HOMBRE,PL,BC,TIJUANA,HOSPITALIZADO,2020-04-02,2020-03-26,...,SI,NO,NO,NO ESPECIFICADO,No positivo SARS-CoV-2,NO ESPECIFICADO,México,97,NO,"(2, 4)"
2020-04-13,USMER,SSA,DF,HOMBRE,NE,DF,TLALPAN,HOSPITALIZADO,2020-03-28,2020-03-17,...,NO,NO,NO,NO,Positivo SARS-CoV-2,NO ESPECIFICADO,México,97,NO,"(9, 12)"


In [9]:
print(dataf.columns)
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2"')
#print(dataq1[['ENTIDAD_RES','MUNICIPIO_RES','FECHA_SINTOMAS','FECHA_INGRESO','FECHA_DEF']])
#dataq1 = dataq1.groupby(['FECHA_SINTOMAS','ENTIDAD_RES'])['SEXO'].agg(['count']).unstack(1) 
#dataq1 = dataq1.groupby(['FECHA_SINTOMAS','ENTIDAD_RES'])['SEXO'].agg(['count']).unstack(1) 

#fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['SEXO']
#fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['SEXO']
#fechadef = dataq1.groupby(['FECHA_DEF']).count()['SEXO']

dataq1[['FECHA_INGRESO','FECHA_SINTOMAS','FECHA_DEF']]
#dataq1.head()

#print(dataq1.head(1))
#dataq1.plot(kind='line', figsize=(15,20))

Index(['ORIGEN', 'SECTOR', 'ENTIDAD_UM', 'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES',
       'MUNICIPIO_RES', 'TIPO_PACIENTE', 'FECHA_INGRESO', 'FECHA_SINTOMAS',
       'FECHA_DEF', 'INTUBADO', 'NEUMONIA', 'EDAD', 'NACIONALIDAD', 'EMBARAZO',
       'HABLA_LENGUA_INDI', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_CON', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'OTRO_CASO', 'RESULTADO', 'MIGRANTE',
       'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI', 'ENTIDAD_MUNICIPIO_RES'],
      dtype='object')


,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF
FECHA_ACTUALIZACION,,,
2020-04-13,2020-04-09,2020-03-28,NaT
2020-04-13,2020-04-06,2020-04-04,NaT
2020-04-13,2020-03-28,2020-03-17,NaT
2020-04-13,2020-04-01,2020-03-26,NaT
2020-04-13,2020-04-07,2020-04-06,NaT
...,...,...,...
2020-04-13,2020-03-31,2020-03-26,NaT
2020-04-13,2020-03-20,2020-03-18,NaT
2020-04-13,2020-04-05,2020-03-31,NaT
